![SPARK_PYSPARK](img/spark_pyspark.jpeg)

# 💫 Apache Spark : le moteur du Big Data moderne

Le **Big Data** désigne des volumes de données trop grands pour être traités sur une seule machine.
Historiquement, on utilisait **Hadoop MapReduce**, mais celui-ci lisait et écrivait souvent sur disque → lent.

**Apache Spark** est né pour répondre à ce problème.
- **Traitement en mémoire (RAM)** → bien plus rapide
- **API haut niveau** (SQL, DataFrame, MLlib, etc.)
- **Multi-langages** : Scala, Java, Python (PySpark), R

Spark repose sur une architecture driver / executors.
- **Driver** : programme principal. Gère le plan d’exécution et collecte les résultats
- **Cluster Manager** : alloue les ressources (CPU, RAM) aux tâches Spark (YARN, Kubernetes, Standalone…)
- **Executors** : exécutent les tâches sur les partitions de données.

Spark distribue automatiquement les données et le calcul sur plusieurs machines.

# ✨ PySpark : l’interface Python de Spark

**PySpark** est l’interface de programmation en Python pour Apache Spark.\
Elle permet d’utiliser la puissance du moteur Spark tout en restant dans un environnement familier pour les développeurs Python.

Concrètement :
- Le code écrit en Python est traduit en instructions que le moteur Spark exécute sur le cluster.
- L’exécution réelle ne se fait pas dans le processus Python, mais dans la JVM de Spark (Java Virtual Machine).
- Cette communication passe par l’API Py4J, une passerelle entre Python et Java.

Ainsi, PySpark combine :
- la simplicité du Python (syntaxe, intégration avec pandas, notebooks, etc.)
- la scalabilité de Spark, capable de traiter des téraoctets de données sur des centaines de noeuds.

# ⌨️ Hands-On

**Objectifs**  
- Comprendre comment démarrer une `SparkSession` en local  
- Manipuler des DataFrames PySpark (lecture, inspection, transformations)  
- Comprendre `lazy evaluation`, `action`, `job / stage / task`  
- Sauvegarder/relire des données (CSV, Parquet)

## 1. Démarrer Spark en local

Avant de pouvoir manipuler des données avec PySpark, il faut créer une session Spark.\
Cette session est le point d’entrée principal vers toutes les fonctionnalités de Spark (lecture de données, SQL, transformations, machine learning, etc.).

En pratique, on la crée à l’aide de la classe **SparkSession** qui permet de configurer et d’initialiser le moteur Spark.\
Une fois cette cellule exécutée, Spark démarre un contexte local et affiche des informations dans la console.
C’est le signe que le driver est prêt à exécuter des tâches et à communiquer avec d’éventuels executors.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("IntroSparkLocal") \
    .master("local[*]") \
    .getOrCreate()

# SparkSession.builder crée un objet constructeur pour définir les paramètres de la session.
# .appName("IntroSparkLocal") définit un nom d’application visible dans l’interface Spark (utile pour identifier ton job).
# .master("local[*]") indique le mode d’exécution :
#       - local signifie que le calcul se fera sur la machine locale.
#       - [*] indique que Spark utilisera tous les cœurs CPU disponibles.
# .getOrCreate() crée une nouvelle session Spark si elle n’existe pas encore, ou récupère celle déjà active.

Aussi, Spark démarre une interface web accessible à l’adresse suivante : http://localhost:4040

Cette interface permet de :
- visualiser les jobs Spark en cours ou terminés
- consulter les stages et tasks
- analyser les plans d’exécution
- suivre l’utilisation des ressources locales (mémoire, CPU)

### 🧭 Exploration recommandée

Ouvrir l’URL http://localhost:4040 dans le navigateur.
Parcourir les différentes sections pour comprendre comment Spark gère un traitement distribué :
- Jobs : résumé des traitements lancés,
- Stages : décomposition logique des jobs,
- Tasks : exécution concrète sur les partitions de données,
- Storage : aperçu du cache et des données en mémoire.

Prendre quelques minutes pour parcourir cette interface avant de continuer car elle aidera à visualiser ce qui se passe “sous le capot” à chaque fois qu'une opération est exécutée dans PySpark.

## 2. Charger le dataset de test :  `employees.csv`

Pour travailler concrètement avec Spark, nous allons charger un petit dataset de test nommé `employees.csv`.\
Ce fichier contient des informations simples sur des employés : identifiant, nom, âge, département, salaire et années d’expérience.

Dans PySpark, la lecture d’un fichier CSV s’effectue via la méthode `spark.read.csv()`.
Cette méthode renvoie un **DataFrame Spark,** c’est-à-dire une **structure tabulaire distribuée** (semblable à un tableau SQL ou à un DataFrame Pandas, mais répartie sur plusieurs noeuds).

In [ ]:
# Charge le fichier csv
# header=True indique que la première ligne contient les noms des colonnes
# inferSchema=True demande à Spark de déduire automatiquement les types de données sinon toutes les colonnes seraient lues comme des chaînes de caractères
df = spark.read.csv("./data/employees.csv", header=True, inferSchema=True)

# Affiche la structure (schéma) du DataFrame, c’est-à-dire les noms de colonnes et leurs types 
df.printSchema()

🔍 **Détaillons ce qu’il se passe en arrière-plan.**

Lors de la lecture du CSV :
1. Le driver Spark reçoit la requête de lecture du fichier csv. Il construit un plan logique pour lire le fichier CSV. Le plan logique est une représentation interne (sous forme d’un graphe `DAG`) qui décrit les étapes nécessaires pour obtenir le résultat final : *"quel fichier lire"*, *"avec quel format (ici CSV)"*, *"quelles colonnes ou types de données"*, *"quelles transformations appliquer (filtres, jointures, agrégations, etc.)"*\
Mais à ce stade, **rien n’est encore exécuté**. Spark se contente de noter l’**intention du programme**.
2. Quand une action est déclenchée (.show(), .count(), etc.), le driver envoie les tâches aux executors.
3. Les executors lisent réellement les données (leurs partitions), exécutent les transformations, et renvoient leurs résultats partiels au driver. (En mode local, les executors sont simplement des threads dans le même processus JVM que le driver.)
4. Le driver agrège ces résultats et les renvoie au programme Python.

Même pour un petit CSV local, le mécanisme est le même que sur un cluster : Spark applique toujours un modèle distribué. Cela rend le passage à grande échelle quasiment transparent pour le code.

### 2.1. Inspection de base

Avant de transformer les données, il est souvent utile de vérifier la structure du DataFrame.\
L'opération est immédiate : elle interroge seulement les métadonnées du DataFrame (pas les données elles-mêmes).

In [ ]:
df.columns

### 2.2. Lazy Evaluation

⚠️ Contrairement à df.columns, les instructions suivantes déclenchent des actions Spark.\
Le moteur Spark exécute réellement des jobs distribués pour lire les données, parcourir toutes les partitions et agréger les résultats partiels.

🔍 **Détaillons ce qu’il se passe en arrière-plan.**
- `df.count()` parcourt l’ensemble du DataFrame pour compter le nombre de lignes. Pour cela : \
Spark crée un job composé de plusieurs stages et tasks, chacun exécuté par les executors sur une partition de données.\
Une fois terminées, les executors envoient leurs résultats partiels (leurs counts locaux) au driver.\
Le driver additionne ces valeurs pour obtenir le compte global du DataFrame,
puis renvoie le résultat final au programme Python.
- `df.show(5)` demande à Spark d’afficher les 5 premières lignes du DataFrame.
Cela nécessite de lire physiquement les données (au moins partiellement) → Spark déclenche donc aussi un job distribué.

In [ ]:
display(df.count())
display(df.show(5))

## 3. Sélections, filtres, tris

Dans Spark, les DataFrames offrent une interface haut niveau pour manipuler les données de manière déclarative, un peu comme en SQL.
Les opérations de sélection, filtrage et tri sont des transformations qui **ne modifient pas les données originales** : elles produisent de **nouveaux DataFrames**.

💡 Chaque transformation crée un nouveau plan logique dans Spark (**aucune lecture ou exécution réelle n’a lieu** tant qu’une action, comme .show() ou .count(), n’est pas appelée).\
Ainsi, même un simple tri ou filtre suit la même logique de planification distribuée que sur un cluster.

In [ ]:
from pyspark.sql.functions import col

# Sélection de colonnes
df_sel = df.select("name", "age", "department")
df_sel.show(5)

# Filtres
employees_filtered = df_sel.filter(col("age") >= 30) # Ne conserver que les employés d'un certain âge
employees_filtered = employees_filtered.orderBy(col("age").desc())
employees_filtered.show(5)

### 3.1. Spark SQL

Il est possible d’écrire directement du ***SQL*** grâce à ***Spark SQL***, un module intégré à Apache Spark qui permet d’interroger les données via des requêtes SQL standard.\
Pour cela, il suffit d’enregistrer un DataFrame comme vue temporaire à l’aide de `createOrReplaceTempView()`. Cette vue agit comme une table virtuelle accessible uniquement durant la session en cours.\
On peut ensuite exécuter des requêtes SQL sur cette vue avec la méthode `spark.sql()`, comme dans l’exemple ci-dessous, qui fait exactement la même chose que le job précédent.

In [ ]:
# Création ou remplacement d'une vue temporaire à partir du DataFrame
df.createOrReplaceTempView("employees")

# Requête SQL exécutée via Spark SQL
result = spark.sql("""
    SELECT department,
           AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
""")

# Affichage du résultat
result.show()

### 🧩 Exercices

> Essayez de modifier et d’expérimenter :
> - sélectionnez d’autres colonnes avec .select().
> - changez la condition du filtre (age < 25, department == "HR", etc.).
> - triez par une autre colonne, ou combinez plusieurs tris
> - ajoutez plusieurs conditions avec les opérateurs logiques `&` (et), `|` (ou)

In [ ]:
# Votre code ici

> Pensez à tester aussi Spark SQL dans cet exercice.

In [ ]:
# Votre code ici

## 4. Nettoyage des données

Dans Spark, nous pouvons appliquer des fonctions de transformation directement sur les colonnes d’un DataFrame de manière distribuée.

Dans l’exemple ci-dessous :
- nous normalisons les noms de département
- nous traitons les valeurs manquantes

💡 Ces opérations sont aussi "lazy" : elles ne modifient pas les données tant qu’aucune action n’est exécutée.

In [ ]:
from pyspark.sql.functions import when, trim, upper

# Normaliser le nom du département: trim + upper
df_sel = df
df_clean = df_sel.withColumn("department_norm", upper(trim(col("department"))))

# Gérer les valeurs manquantes d'âge et de salaire
df_clean = df_clean.withColumn("age", when(col("age").isNull(), 0).otherwise(col("age")))
df_clean = df_clean.na.fill({"salary": 0.0})

df_clean.select("employee_id","name","department","department_norm","age","salary").show(5)

### 4.1. Créer des colonnes calculées

La préparation des données consiste aussi à créer de nouvelles colonnes dérivées à partir des colonnes existantes.\
Dans Spark, cela se fait avec la méthode `withColumn()`, qui permet d’ajouter une colonne calculée sans modifier le DataFrame original.

👉 Si le nom de colonne existe déjà, la colonne est remplacée (écrasée) par la nouvelle version.\
👉 Si le nom n’existe pas, une nouvelle colonne est simplement ajoutée au DataFrame, c'est le cas dans l'exemple suivant.

In [ ]:
from pyspark.sql.functions import avg

# Catégoriser les revenus
df_feat = df_clean.withColumn(
    "salary_band",
    when(col("salary") < 3000, "LOW").when(col("salary") < 6000, "MID").otherwise("HIGH")
)

df_feat.show(5)


## 5. Regroupement et agrégation

La phase d’agrégation permet de résumer, regrouper et interpréter les informations issues de grands volumes de données.\
Avec PySpark, ces opérations s’effectuent facilement grâce aux fonctions de groupement (`groupBy`), de comptage (`count`), et d’agrégation (`agg`).

In [ ]:
# Nombre d'employés par catégorie de revenus
df_feat\
    .groupBy("salary_band")\
    .count()\
    .orderBy(col("count").desc())\
    .show()

In [ ]:
from pyspark.sql.functions import round as sround

# Analyse des salaires moyens par département 
dept_stats = df_feat\
                .groupBy("department_norm")\
                .agg(sround(avg("salary"),2).alias("avg_salary"))\
                .orderBy(col("avg_salary").desc())

dept_stats.show()


### 🧩 Exercices

> Comment Spark découpe les transformations ci-dessus en **stages** ?  
> Ouvrez l'onglet **SQL / DAG** dans l'UI Spark puis analysez les informations présentées.


## 6. Sauvegarde de résultats (CSV / Parquet)

Une fois les traitements réalisés nous pouvons sauvegarder les résultats pour les réutiliser ou les partager.\
PySpark permet d’écrire facilement un DataFrame dans différents formats, notamment CSV et Parquet (format binaire optimisé pour le Big Data).

- Le format CSV est universel mais peu efficace sur de gros volumes, car il ne conserve pas les types de données.
- Le format Parquet, lui, conserve le schéma, compresse les données et accélère les lectures : il est donc privilégié dans les environnements distribués.

### 🔎 Format Parquet
Pour aller plus loin concernant le format Parquet, voici quelques ressources :
- 📘 [Documentation officielle Apache Parquet](https://parquet.apache.org/docs/)  
- 🎥 [Qu’est-ce qu'une base de données orientée colonne ?](https://www.youtube.com/watch?v=1MnvuNg33pA)

In [ ]:
# Écriture en CSV (dossier de sortie partitionné)
dept_stats.write.mode("overwrite").csv("./outputs/dept_stats_csv")

# Écriture en Parquet (conserve le schéma, plus efficace)
dept_stats.write.mode("overwrite").parquet("./outputs/dept_stats_parquet")

# Relecture du Parquet
reloaded = spark.read.parquet("./outputs/dept_stats_parquet")
reloaded.printSchema()
reloaded.show()


## 7. RDD (Resilient Distributed Dataset)

Jusqu’à présent, vous avez manipulé des **DataFrames** : une interface structurée, avec des colonnes nommées, des types de données, et des opérations familières (sélection, groupBy, agg, etc.).\
Les DataFrames sont très pratiques, car ils permettent d’écrire du code lisible et optimisé grâce au moteur **Catalyst** de Spark.

Mais sous cette interface haut niveau se cache la brique fondamentale de Spark : le **RDD (Resilient Distributed Dataset)**.\
Comprendre ce concept vous permettra de mieux saisir comment Spark distribue, exécute et reconstruit vos données à grande échelle.

Un RDD est une collection d’objets distribuée à travers le cluster Spark.
- **Resilient** : il peut être reconstruit automatiquement en cas de panne.
- **Distributed** : les données sont découpées en partitions réparties sur plusieurs nœuds.
- **Dataset** : c’est simplement un ensemble d’éléments (lignes, objets, etc.).

Autrement dit, un RDD est une collection répartie sur plusieurs machines, que Spark sait manipuler en parallèle.

### 7.1. Manipulation de base

In [ ]:
sc = spark.sparkContext

# Exemple d'un RDD à partir d’une liste locale
# Spark crée un RDD contenant ces 5 éléments.
rdd = sc.parallelize([1, 2, 3, 4, 5])

In [ ]:
# Rien n’est exécuté tant qu’on ne déclenche pas une action
display(rdd.collect())

In [ ]:
# On peut transformer les données
rdd2 = rdd.map(lambda x: x * 2)
display(rdd2.collect())

#### 🧩 Exercices

> Soit la liste de produits :

```bash
ventes = [
    ("chaise", 200),
    ("table", 450),
    ("lampe", 100),
    ("chaise", 150),
    ("table", 300),
    ("lampe", 80)
]
```

> - Créez un RDD à partir de cette liste.
> - Récupérez les produits dont le prix dépasse 100€
> - Calculez le CA total par produit (possible d'utiliser .reduceByKey())
> - Triez par CA décroissant (possible d'utiliser .sortBy())

In [ ]:
# Votre code ici

### 7.2. Manipulation de fichiers

In [ ]:
# Spark permet de lire et de manipuler des fichiers sous forme de RDD.
employees_rdd = sc.textFile("./data/employees.csv")
employees_rdd.collect()

#### 🧩 Exercices

> Ne récupérer que les employés du département IT.\
> Indice : .map(), .filter(), .split()

```bash
Exemple :

[['22', 'Alice Bernard', '33.0', 'IT', '6231.15', '10'],
 ['32', 'Hugo Girard', '58.0', 'IT', '8616.31', '20'],
 ['40', 'Emma Morel', '42.0', 'IT', '6115.84', '10'],
 ['72', 'Tom Petit', '41.0', 'IT', '5449.79', '7'],
 ['83', 'Ines Durand', '54.0', 'IT', '7555.4', '13']]
 ```

In [ ]:
# Votre code ici

## 8. Approfondir

Ce notebook propose une **introduction pratique** à Apache Spark et PySpark.  
Il ne couvre pas toutes les subtilités du framework, notamment celles liées à l’**optimisation des performances**, la **gestion des clusters**, ou les **fonctionnalités avancées**, MLlib, ou Structured Streaming.  

Pour compléter votre apprentissage, il est **fortement recommandé de poursuivre l’exploration** à travers d'autres ressources.\
Il existe **de nombreuses ressources de qualité disponibles en ligne**, il suffit de chercher un peu pour trouver des explications, tutoriels et exemples adaptés à votre niveau.

L’objectif de cet approfondissement est de comprendre Apache Spark dans son contexte Big Data, non pas comme un simple outil, mais comme une évolution et une réponse aux limites de Hadoop.
Il s’agit d’identifier les différences d’architecture, de performance et de paradigme de calcul, tout en comprenant comment ces deux technologies peuvent coexister et se compléter.

#### Axes d'approfondissement
> Architecture de Spark
- Rôles du Driver, des Executors et du Cluster Manager
- Traitement en mémoire et exécution des DAG (Directed Acyclic Graph)
- Modes d’exécution : Local vs Cluster

> Comparaison Hadoop / Spark
- Stockage : HDFS et compatibilité avec Spark
- Gestion des ressources : rôle de YARN
- Modèle de calcul : batch vs in-memory

> Écosystème et usages
- Modules : Spark SQL, MLlib, Streaming, GraphX


#### Ressources officielles
- [Documentation officielle de Spark](https://spark.apache.org/docs/latest/) — la source principale pour comprendre les modules (SQL, MLlib, Streaming, GraphX).  
- [API PySpark sur spark.apache.org](https://spark.apache.org/docs/latest/api/python/) — référence complète des fonctions disponibles.  
- [Apache Spark sur GitHub](https://github.com/apache/spark) — pour explorer le code source et les évolutions du projet.  

#### Tutoriels et cours gratuits
- [Spark Tutorial (DataCamp)](https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark) — bonnes bases pour pratiquer avec PySpark.  
- [YouTube – “Apache Spark Full Course” by Simplilearn](https://www.youtube.com/watch?v=_C8kWso4ne4) — formation vidéo complète et vulgarisée.   